# Create databases of RRSG/qMRSG Challenge data

## Imports

In [1]:
from pathlib import Path
import pandas as pd
import json
import nibabel as nib
import numpy as np

from src.database import *
from make_pooled_datasets import *

## Configurations

In [2]:
configFile = Path('configs/3T_NIST_T1maps.json')
roiConfigFile = Path('configs/3T_NIST_T1maps_rois.json')
data_folder_name = '3T_NIST_T1maps'
roi_folder_name = '3T_NIST_T1maps_rois'
output_filename = data_folder_name + '_database' 
output_folder = Path('databases/')

## Download datasets

In [3]:
if not Path(data_folder_name).exists():
    make_pooled_dataset(configFile, data_folder_name)

if not Path(roi_folder_name).exists():
    make_pooled_dataset(roiConfigFile, roi_folder_name)

https://osf.io/7k96d/download/
100% [............................................................................] 568226 / 568226

https://osf.io/46nd9/download/
100% [............................................................................] 312539 / 312539

https://osf.io/nk6ae/download/
100% [............................................................................] 279647 / 279647

https://osf.io/t5y28/download/
100% [..........................................................................] 1677684 / 1677684

https://osf.io/vw24u/download/
100% [............................................................................] 292703 / 292703

https://osf.io/sjy5g/download/
100% [..........................................................................] 2234880 / 2234880

https://osf.io/nax9z/download/
100% [............................................................................] 384589 / 384589

https://osf.io/bfh4m/download/
100% [.......................................

## Create database

In [4]:
df = create_database(configFile, data_folder_name, roiConfigFile, roi_folder_name)

## Save database

In [5]:
df.to_excel(output_folder / (output_filename + '.xlsx')) # To excel
df.to_pickle(output_folder / (output_filename + '.pkl')) # To pickle (Python object)
df.to_csv(output_folder / (output_filename + '.csv')) # To csv

## Database manipulation demo

### View entire database

In [6]:
df

,OSF dataset,OSF link,NIFTI filename,Data type,contact,site name,MRI vendor,MRI version,MRI field,sample type,...,T1 - NIST sphere 5,T1 - NIST sphere 6,T1 - NIST sphere 7,T1 - NIST sphere 8,T1 - NIST sphere 9,T1 - NIST sphere 10,T1 - NIST sphere 11,T1 - NIST sphere 12,T1 - NIST sphere 13,T1 - NIST sphere 14
id,,,,,,,,,,,,,,,,,,,,,
1.001,matthewgrechsollars_ICL_NIST,https://osf.io/7k96d/download/,20200121_matthewgrechsollars_ICL_NIST/20200121...,Magnitude,Matthew Grech-Sollars,Imperial College London,Siemens,Prisma VE11C,3.0,NIST,...,"[486.3, 485.5, 487.2, 489.1, 492.1, 494.1, 488...","[345.5, 346.6, 344.9, 345.4, 343.7, 352.4, 342...","[245.8, 248.4, 245.6, 243.3, 238.6, 236.1, 240...","[175.1, 173.4, 178.8, 180.8, 181.3, 177.8, 176...","[183.8, 181.6, 185.3, 184.6, 118.4, 118.1, 116...","[108.6, 96.3, 81.1, 72.4, 90.5, 100.2, 94.6, 8...","[101.0, 86.5, 2.5, 1.1, 2.6, 56.5, 68.2, 71.5,...","[4.0, 2.8, 4.2, 77.1, 69.6, 1.2, 80.8, 63.0, 2...","[75.6, 80.7, 79.2, 72.4, 106.0, 90.5, 3.7, 3.0...","[57.9, 71.3, 1.2, 2.8, 1.5, 97.1, 4.4, 84.3, 2..."
1.002,matthewgrechsollars_ICL_NIST,https://osf.io/7k96d/download/,20200121_matthewgrechsollars_ICL_NIST/20200121...,Complex,Matthew Grech-Sollars,Imperial College London,Siemens,Prisma VE11C,3.0,NIST,...,"[486.3, 485.5, 487.2, 489.1, 492.3, 494.1, 488...","[345.5, 346.6, 344.9, 345.4, 343.7, 352.4, 342...","[245.8, 248.4, 245.6, 243.3, 238.6, 236.1, 240...","[175.1, 173.4, 178.8, 180.8, 181.3, 177.8, 176...","[127.7, 128.7, 129.9, 128.7, 118.4, 118.1, 116...","[98.7, 96.4, 81.1, 72.4, 90.6, 91.6, 86.8, 81....","[101.0, 86.5, 6.2, 10.0, 3.3, 58.2, 70.5, 73.3...","[3.3, 1.1, 6.7, 76.9, 69.8, 10.0, 80.8, 71.9, ...","[75.5, 81.0, 79.2, 72.3, 106.1, 90.3, 3.3, 1.9...","[89.1, 113.3, 5.9, 4.0, 1.5, 96.7, 3.4, 84.5, ..."
2.001,siyuanhu_casewestern_NIST,https://osf.io/46nd9/download/,20200124_siyuanhu_casewestern_NIST/20200124_si...,Magnitude,Siyuan Hu,"Biomedical Engineering, Case Western Reserve U...",Siemens,Skyra,3.0,NIST,...,"[411.8, 405.8, 404.0, 403.3, 399.1, 397.0, 398...","[243.9, 257.4, 265.5, 260.9, 266.9, 264.5, 268...","[194.7, 99.2, 102.7, 103.9, 97.1, 118.0, 132.9...","[14.1, 9.5, 10.0, 1.4, 3.0, 4.9, 14.0, 14.0, 1...","[3.3, 10.0, 3.0, 10.6, 2.7, 13.2, 13.0, 2.0, 6...","[1.2, 10.7, 1.2, 3.3, 3.3, 3.6, 9.5, 97.4, 2.6...","[72.6, 98.0, 9.5, 97.6, 66.4, 59.3, 70.3, 83.5...","[83.1, 1.3, 71.6, 77.7, 1.0, 96.8, 94.5, 84.6,...","[9.5, 2.7, 115.4, 93.9, 101.7, 114.1, 108.8, 8...","[85.8, 104.3, 1.2, 1.8, 127.8, 1.3, 124.5, 5.3..."
3.001,iveslevesque_muhc_mgh_NIST,https://osf.io/nk6ae/download/,20200203_iveslevesque_muhc_mgh_NIST/20200203_i...,Magnitude,Ives Levesque,McGill University Health Centre - Montreal Gen...,Siemens,syngo MR E11,3.0,NIST,...,"[441.5, 444.1, 447.8, 439.5, 461.8, 466.1, 468...","[343.7, 331.3, 326.6, 350.0, 334.6, 328.2, 346...","[233.2, 239.1, 241.6, 237.5, 237.0, 243.1, 240...","[172.9, 177.2, 183.0, 180.9, 177.0, 175.1, 177...","[182.6, 181.7, 180.5, 127.3, 119.7, 112.3, 117...","[111.7, 99.3, 92.3, 86.1, 113.5, 116.2, 99.2, ...","[47.2, 9.5, 84.5, 95.7, 70.7, 1.0, 75.3, 86.0,...","[3.0, 1.8, 1.2, 5.6, 74.3, 9.5, 46.4, 96.2, 2....","[9.6, 2.0, 2.0, 90.5, 97.2, 107.1, 3.3, 2.7, 1...","[97.6, 1.3, 10.0, 107.2, 132.8, 9.6, 72.2, 153..."
4.001,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200204_mrel_usc_GE3T_...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,NIST,...,"[475.4, 483.3, 483.7, 481.8, 485.6, 482.1, 479...","[351.5, 344.0, 341.9, 338.0, 338.5, 343.8, 343...","[241.4, 237.1, 239.9, 243.0, 241.4, 241.9, 239...","[167.0, 169.7, 172.5, 171.0, 166.8, 168.0, 173...","[61.5, 112.4, 117.0, 114.3, 117.3, 110.7, 126....","[69.5, 89.5, 90.8, 88.0, 91.1, 66.0, 90.3, 87....","[55.0, 63.8, 62.1, 66.7, 66.5, 1.3, 41.6, 71.0...","[66.3, 10.0, 63.0, 74.6, 86.5, 2.5, 64.4, 74.9...","[1.0, 69.1, 50.9, 2.5, 1.9, 94.2, 6.1, 3.8, 2....","[60.7, 58.7, 47.6, 100.5, 40.0, 2.0, 1.6, 3.2,..."
4.002,mrel_usc_NIST,https:/

### View a column values for a specific database ID (i.e. row)

In [7]:
df.loc[1.001]

OSF dataset                                   matthewgrechsollars_ICL_NIST
OSF link                                    https://osf.io/7k96d/download/
NIFTI filename           20200121_matthewgrechsollars_ICL_NIST/20200121...
Data type                                                        Magnitude
contact                                              Matthew Grech-Sollars
site name                                          Imperial College London
MRI vendor                                                         Siemens
MRI version                                                   Prisma VE11C
MRI field                                                              3.0
sample type                                                           NIST
phantom version                                                        130
phantom serial number                                                  102
phantom temperature                                                  21.89
age                      

### Get the value of a cell entry (row/column intersection)

In [8]:
df.loc[1.001]['T1 - genu (WM)']